## Generating different colors of noise

This notebook demonstrates how to generate different colors of noise. Different colors of noise are described here (with audio samples): https://en.wikipedia.org/wiki/Colors_of_noise .

Some of the code below is heavily inspired by the noise generators in python-acoustics.

In [ ]:
import numpy as np

np.random.seed(1337)

%matplotlib inline
import matplotlib.pyplot as plt

import scipy.signal

In [ ]:
sample_rate = 16000
samples_to_generate = 16000

In [ ]:
def to_16bit(samples):
    # assume +1 corresponds to +32767 and -1 corresponds to -32767
    # (note that we don't use -32768)
    # 
    # does not convert the samples to int16 for the moment
    
    return np.clip(32767 * samples, -32767, +32767)

In [ ]:
def normalize(samples):
    """normalizes a sample to unit standard deviation (assuming the mean is zero)"""
    std = samples.std()
    if std > 0:
        return samples / std
    else:
        return samples

In [ ]:
def _gen_colored_noise(spectral_shape):
    # helper function generating a noise spectrum
    # and applying a shape to it
    flat_spectrum = np.random.normal(size = samples_to_generate // 2 + 1) + \
            1j * np.random.normal(size = samples_to_generate // 2 + 1)

    return normalize(np.fft.irfft( flat_spectrum * spectral_shape).real)
        

def gen_noise(color):
    
    assert samples_to_generate % 2 == 0
    
    if color == 'white':
        # flat in frequency
        
        # note that this needs to be normalized because
        # with std = 1 many samples will be outside +1/-1
        return np.random.normal(size = samples_to_generate)
    
    spectrum_len = samples_to_generate // 2 + 1
    
    if color == 'pink':
        return _gen_colored_noise(1. / (np.sqrt(np.arange(spectrum_len) + 1.)))
        
    elif color == 'blue':
        return _gen_colored_noise(np.sqrt(np.arange(spectrum_len)))
    
    elif color == 'brown' or color == 'red':
        return _gen_colored_noise(1. / (np.arange(spectrum_len) + 1))

    elif color == 'violet' or color == 'purple':
        return _gen_colored_noise(np.arange(spectrum_len))
    
    else:
        raise Exception("unsupported noise color %s" % color)

generate an example for each of the supported colors and inspect it visually

In [ ]:
noise = {}

for color in ('white',   # flat spectrum
              'pink',    # -3dB (factor 0.5)  per octave / -10 dB (factor 0.1) per decade
              'blue',    # +3dB (factor 2)    per octave / +10 dB (factor 10) per decade
              'brown',   # -6dB (factor 0.25) per octave / -20 dB (factor 0.01) per decade
              'violet'): # +6dB (factor 4)    per octave / +20 dB (factor 100) per decade
    
    noise[color] = to_16bit(gen_noise(color) / 4)
    
    plt.figure(figsize = (15,7))
    
    plt.subplot(1,2,1)
    plt.plot(noise[color])
    plt.grid()
    plt.title("%s noise" % color)
    plt.ylabel('amplitude')
    
    # plot spectral power density
    plt.subplot(1,2,2)
    freqs, spec = scipy.signal.welch(noise[color], fs = sample_rate)
    
    # normalize to middle of spectrum
    # freqs -= freqs[len(freqs) // 2]
    
    plt.loglog(freqs, spec)
    plt.gca().minorticks_on()
    plt.grid(True, which = 'both')
    # plt.ylim(ymin = 0)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('power spectral density')

listen to the generated samples

In [ ]:
import IPython

In [ ]:
IPython.display.Audio(noise['white'], rate = sample_rate)

In [ ]:
IPython.display.Audio(noise['pink'], rate = sample_rate)

In [ ]:
IPython.display.Audio(noise['blue'], rate = sample_rate)

In [ ]:
IPython.display.Audio(noise['brown'], rate = sample_rate)

In [ ]:
IPython.display.Audio(noise['violet'], rate = sample_rate)